In [1]:
import os
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from datetime import datetime

In [3]:
# train data 
TRAIN_DATA_PATH = os.getenv("train_data_path")
TRAIN_DATA_DESCRIPTION = os.getenv("train_data_description")

# model data
MODEL_PATH = os.getenv("model_path")
MODEL_ARCHITECTURE = os.getenv("model_architecture")
MODEL_ID = os.getenv("model_id")
MODEL_METADATA_PATH = MODEL_PATH + "metadata.yaml"

# model hyperparameters
VECTOR_SIZE = os.getenv("vector_size")
WINDOW = os.getenv("window")
MIN_COUNT = os.getenv("min_count")
WORKERS = os.getenv("workers")

In [4]:
# train data 
print(TRAIN_DATA_PATH)
print(TRAIN_DATA_DESCRIPTION)

# model data
print(MODEL_PATH)
print(MODEL_ARCHITECTURE)
print(MODEL_ID)
print(MODEL_METADATA_PATH)

# model hyperparameters
print(VECTOR_SIZE)
print(WINDOW)
print(MIN_COUNT)
print(WORKERS)

/veld/input/de_wiki_cleaned.csv
german wikipedia, subset, csv
/veld/output/m4/
word2vec_v1
m4
/veld/output/m4/metadata.yaml
300
3
1
-1


In [5]:
TRAIN_DATA_PATH = "/veld/input/data.csv"
MODEL_PATH = "/veld/output/" + os.getenv("MODEL_NAME")
MODEL_METADATA_PATH = "/veld/output/metadata.yaml"
TRAIN_DATA_NAME = "german wikipedia, csv"
VECTOR_SIZE = 200
WINDOW = 3
MIN_COUNT = 1
WORKERS = 10

In [6]:
class Iterable:

    def __init__(self, csv_file_path):
        self.df = pd.read_csv(csv_file_path)
    
    def __iter__(self):
        for line in self.df.iterrows():
            yield simple_preprocess(line[1]["text"])

iter = Iterable(TRAIN_DATA_PATH)

IsADirectoryError: [Errno 21] Is a directory: '/veld/input/data.csv'

In [ ]:
time_start = datetime.now()
model = gensim.models.Word2Vec (iter,
                                vector_size=VECTOR_SIZE,
                                window=WINDOW,
                                min_count=MIN_COUNT,
                                workers=WORKERS)
time_end = datetime.now()
duration = (time_end - time_start).seconds / 60
model.save(MODEL_PATH)

In [ ]:
# calculate size of training data
def calc_size(file):
    size = os.path.getsize(file)
    for unit in ["","KB","MB","GB","TB"]:
        if abs(size) < 1024.0:
            return f"{round(size, 1)} {unit}"
        size /= 1024.0
train_data_size = calc_size(TRAIN_DATA_PATH)
model_data_size = calc_size(MODEL_PATH)

# calculate hash of training data
train_data_md5_hash = subprocess.run(["md5sum", TRAIN_DATA_PATH], capture_output=True, text=True)
train_data_md5_hash = train_data_md5_hash.stdout.split()[0]


# aggregate into metadata dictionary
metadata = {
    "train_data_name": TRAIN_DATA_NAME,
    "train_data_size": train_data_size,
    "train_data_md5_hash": train_data_md5_hash,
    "training_vector_size": VECTOR_SIZE,
    "window": WINDOW,
    "min_count": MIN_COUNT,
    "workers": WORKERS,
    "training_duration (minutes)": round(duration, 1),
    "model_data_size": model_data_size,
}

# write to yaml
with open(MODEL_METADATA_PATH, "w") as f:
    # iteration over dictionary to ensure the yaml writer respects the order
    for k, v in metadata.items():
        yaml.dump({k: v}, f)